In [21]:
from bosonicplus.base import State
from thewalrus.symplectic import beam_splitter, expand, squeezing, xxpp_to_xpxp
import numpy as np
from time import time

from strawberryfields.backends.gaussianbackend.gaussiancircuit import GaussianModes
from strawberryfields.backends.bosonicbackend.bosoniccircuit import BosonicModes

In [22]:
# Comparing the time it takes to apply one single-mode and one two-mode gate in bosonicplus vs strawberryfield's backends. 

nmodes = 1000

t1 = time()
test = State(nmodes)
S = expand(squeezing(1,0), 1, nmodes)
S = expand(beam_splitter(np.pi/4, 0), [0,1], nmodes)@S
test.apply_symplectic(xxpp_to_xpxp(S))

covbp1 = test.covs
t2 = time()

print('bosonicplus time:', t2-t1)

t1 = time()

test = State(nmodes)
S = squeezing(1,0)
test.apply_symplectic_fast(S, [1])
S = xxpp_to_xpxp(beam_splitter(np.pi/4,0))
test.apply_symplectic_fast(S, [0,1])
t2 = time()
covbp2 = test.covs
print('bosonicplus fast:', t2-t1)

print('Same cov?', np.allclose(covbp1,covbp2))
#The gaussian backend is faster than bosonicplus and bosonicbackend 
#because each gate is hard coded to change the elements of the N and M mat.
#(See https://github.com/XanaduAI/strawberryfields/blob/master/strawberryfields/backends/gaussianbackend/gaussiancircuit.py)

t0 =time()
gs = GaussianModes(nmodes)
gs.squeeze(1,0,1)
gs.beamsplitter(np.pi/4, 0, 0,1)
cov = gs.scovmatxp()
t1 = time()
print('sf gaussian backend time:', t1-t0)

t0 = time()
bs = BosonicModes(nmodes)
bs.squeeze(1,0,1)
bs.beamsplitter(np.pi/4, 0, 0,1)
t1 = time()
covbs = bs.covs
print('sf bosonic backend time: ', t1-t0)



bosonicplus time: 0.5469448566436768
bosonicplus fast: 0.041207075119018555
Same cov? True
sf gaussian backend time: 0.07728219032287598
sf bosonic backend time:  2.7756998538970947
